In [1]:
import os, sys
import datetime
from datetime import datetime, date, timedelta
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas.plotting import scatter_matrix
from pandas_profiling import ProfileReport
import mglearn
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
%config InlineBackend.figure_format = 'svg'
from mpl_toolkits.mplot3d import Axes3D, axes3d
from scipy import sparse
import scipy.stats as ss
import scipy.cluster.hierarchy as sch
import pingouin as pg
import spacy, nltk
from konlpy.tag import Okt

c:\Users\admin\anaconda3\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.5.2, the latest is 0.5.3.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


In [22]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [2]:
data_df = pd.read_csv(os.path.join(mglearn.datasets.DATA_PATH, 'adult.data'), header=None, index_col=False, 
names = ['age', 'workclass', 'fnlwgt', 'education',  'education-num',
           'marital-status', 'occupation', 'relationship', 'race', 'gender',
           'capital-gain', 'capital-loss', 'hours-per-week', 'native-country',
           'income'])
data = data_df.loc[:, ['age', 'workclass', 'education', 'gender', 'hours-per-week', 'occupation', 'income']]
data.head()

,age,workclass,education,gender,hours-per-week,occupation,income
0,39,State-gov,Bachelors,Male,40,Adm-clerical,<=50K
1,50,Self-emp-not-inc,Bachelors,Male,13,Exec-managerial,<=50K
2,38,Private,HS-grad,Male,40,Handlers-cleaners,<=50K
3,53,Private,11th,Male,40,Handlers-cleaners,<=50K
4,28,Private,Bachelors,Female,40,Prof-specialty,<=50K


In [8]:
pd.concat([data.dtypes, data.nunique()], axis=1)

,0,1
age,int64,73
workclass,object,9
education,object,16
gender,object,2
hours-per-week,int64,94
occupation,object,15
income,object,2


In [10]:
# dtypes에서 확인할 때 object일 경우에는 자동으로 변환
# columns 파라미터 추가로 변환하고 싶은 열 설정 가능
data_dummies = pd.get_dummies(data)
list(data.columns), list(data_dummies.columns)

(['age',
  'workclass',
  'education',
  'gender',
  'hours-per-week',
  'occupation',
  'income'],
 ['age',
  'hours-per-week',
  'workclass_ ?',
  'workclass_ Federal-gov',
  'workclass_ Local-gov',
  'workclass_ Never-worked',
  'workclass_ Private',
  'workclass_ Self-emp-inc',
  'workclass_ Self-emp-not-inc',
  'workclass_ State-gov',
  'workclass_ Without-pay',
  'education_ 10th',
  'education_ 11th',
  'education_ 12th',
  'education_ 1st-4th',
  'education_ 5th-6th',
  'education_ 7th-8th',
  'education_ 9th',
  'education_ Assoc-acdm',
  'education_ Assoc-voc',
  'education_ Bachelors',
  'education_ Doctorate',
  'education_ HS-grad',
  'education_ Masters',
  'education_ Preschool',
  'education_ Prof-school',
  'education_ Some-college',
  'gender_ Female',
  'gender_ Male',
  'occupation_ ?',
  'occupation_ Adm-clerical',
  'occupation_ Armed-Forces',
  'occupation_ Craft-repair',
  'occupation_ Exec-managerial',
  'occupation_ Farming-fishing',
  'occupation_ Handlers-cl

In [14]:
X = data_dummies.iloc[:, :-2]
y = data.iloc[:, -1].replace({' <=50K':0, ' >50K':1})
X.shape, y.shape

((32561, 44), (32561,))

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [21]:
logreg = LogisticRegression(C=100, max_iter=3000).fit(X_train, y_train)
logreg.score(X_train, y_train), logreg.score(X_test, y_test)

(0.8138001638001638, 0.8086230192851002)

In [29]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=15, random_state=0, n_jobs=-1).fit(X_train, y_train)
rf_clf.score(X_train, y_train), rf_clf.score(X_test, y_test)

(0.8620393120393121, 0.8141505957499079)